In [40]:
import pandas as pd
from ethnicolr import census_ln, pred_census_ln,pred_fl_reg_name
import cnfg
from sqlalchemy import create_engine
import time

### Test Case

In [33]:
names = [{'last name': 'smith','first name':'Bob'}, {'last name': 'zhang','first name':'Becky'},{'last name': 'jackson','first name':'randy'}
        ,{'last name': 'kim','first name':'brian'},{'last name': 'dionisio','first name':'sebastian'}]

In [34]:
df = pd.DataFrame(names)
df

,first name,last name
0,Bob,smith
1,Becky,zhang
2,randy,jackson
3,brian,kim
4,sebastian,dionisio


    pctwhite: Percent Non-Hispanic White Alone
    pctblack: Percent Non-Hispanic Black or African American Alone
    pctapi: Percent Non-Hispanic Asian and Native Hawaiian and Other Pacific Islander Alone
    pctaian: Percent Non-Hispanic American Indian and Alaska Native Alone
    pct2prace:Percent Non-Hispanic Two or More Races
    pcthispanic:Percent Hispanic or Latino origin

In [36]:
census_ln(df, 'last name')

,first name,last name,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,Bob,smith,73.35,22.22,0.40,0.85,1.63,1.56
1,Becky,zhang,0.61,0.09,98.16,0.02,0.96,0.16
2,randy,jackson,41.93,53.02,0.31,1.04,2.18,1.53
3,brian,kim,2.60,0.36,94.52,0.03,1.99,0.50
4,sebastian,dionisio,43.14,0.32,39.17,0.00,3.08,14.29


In [37]:
census_ln(df, 'last name', 2010)

,first name,last name,pctwhite,pctblack,pctapi,pctaian,pct2prace,pcthispanic
0,Bob,smith,70.9,23.11,0.5,0.89,2.19,2.4
1,Becky,zhang,0.99,0.16,98.06,0.02,0.62,0.15
2,randy,jackson,39.89,53.04,0.39,1.06,3.12,2.5
3,brian,kim,2.52,0.39,94.47,0.02,1.96,0.65
4,sebastian,dionisio,33.09,(S),42.61,(S),1.71,22.34


In [38]:
pred_census_ln(df, 'last name')

,first name,last name,race,api,black,hispanic,white
0,Bob,smith,white,0.002019,0.247235,0.014485,0.736260
1,Becky,zhang,api,0.997807,0.000149,0.000470,0.001574
2,randy,jackson,black,0.002797,0.528193,0.014605,0.454405
3,brian,kim,api,0.974226,0.000943,0.008638,0.016193
4,sebastian,dionisio,white,0.014506,0.035601,0.022365,0.927528


In [39]:
pred_fl_reg_name(df,'last name','first name')

,first name,last name,race,asian,hispanic,nh_black,nh_white
0,Bob,smith,nh_white,0.008679,0.007090,0.122469,0.861762
1,Becky,zhang,asian,0.963188,0.003856,0.009320,0.023636
2,randy,jackson,nh_white,0.003070,0.012693,0.451494,0.532742
3,brian,kim,asian,0.874900,0.003652,0.007198,0.114250
4,sebastian,dionisio,nh_white,0.004722,0.258723,0.004320,0.732235


###  1% of active users

In [28]:
db_config = cnfg.load("/home/jovyan/work/.fff_config")
redshift_engine = create_engine('postgresql://' + 
                                 db_config['redshift_user'] + ':' + 
                                 db_config['redshift_password'] + '@' +
                                 db_config['redshift_host'] + ':' + 
                                 db_config['redshift_port'] + '/' + 
                                 db_config['redshift_db'])

In [29]:
#Get 1000 sample data
df_sample_query = '''
select distinct last_name,first_name
from dw_user_cube
where account_state = 'active'
limit 1000
'''
df_sample = pd.read_sql_query(df_sample_query, redshift_engine)

In [30]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
last_name     1000 non-null object
first_name    1000 non-null object
dtypes: object(2)
memory usage: 15.7+ KB


In [100]:
t0 = time.time()
df_sample_pred = pred_fl_reg_name(df_sample,'last_name','first_name')
t1 = time.time()
print('1% data processing time',t1-t0)

1% data processing time 9.854230165481567


In [101]:
t0 = time.time()
df_sample_pred_last = pred_census_ln(df_sample,'last_name')
t1 = time.time()
print('1% data processing time',t1-t0)

1% data processing time 9.357678651809692


In [102]:
df_sample_pred = df_sample_pred.iloc[:,0:3]
df_sample_pred_last = df_sample_pred_last.iloc[:,0:3]
df_sample_pred_last = df_sample_pred_last.rename(columns={'race': 'last_race'})
df_sample_pred_last.head()

,last_name,first_name,last_race
0,Daniel,Nichola,white
1,Fielding,Jean,white
2,MALPHRUS,Donna,white
3,Fleming,Stephanie,white
4,Dollard,Kelly,white


In [103]:
df_sample_pred.groupby('race').size()

race
asian        25
hispanic     63
nh_black     41
nh_white    871
dtype: int64

In [104]:
df_sample_pred_last.groupby('last_race').size()

last_race
api          22
black         5
hispanic     79
white       894
dtype: int64

In [105]:
df_sample_pred = df_sample_pred.merge(df_sample_pred_last, how='inner', on=['last_name','first_name'])

In [106]:
df_sample_pred['last_race'].unique()
df_sample_pred['last_race'] = df_sample_pred['last_race'].map({'api':'asian', 'black': 'nh_balck','white':'nh_white','hispanic':'hispanic'})

In [107]:
df_sample_pred[df_sample_pred.race != df_sample_pred.last_race]

,last_name,first_name,race,last_race
9,mutlaq,yasmine,asian,nh_white
11,Davis,Kendel,nh_black,nh_white
23,Mendoza,Kathryn,nh_white,hispanic
33,Marshall-Young,Makenzie,nh_black,nh_white
45,Jackson,Natasha,nh_black,nh_balck
48,Davis,Deidra,nh_black,nh_white
57,Mansilla,Kirsten,nh_white,hispanic
59,Emanuel,Tiffani,nh_black,nh_white
75,Luna,Becky,nh_white,hispanic
88,Pattiwar,Kalyani,asian,nh_white
